In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [3]:
wikipedia.run("Figure skating at the Olympic Games")

"Page: Figure skating at the Olympic Games\nSummary: Figure skating was first contested in the Olympic Games at the 1908 Summer Olympics. Since 1924, the sport has been a part of the Winter Olympic Games.\nMen's singles, ladies' singles, and pair skating have been held most often. Ice dance joined as a medal sport in 1976 and a team event debuted at the 2014 Olympics. Special figures were contested at only one Olympics, in 1908. Synchronized skating has never appeared at the Olympics but aims to be included.\n\n\n\nPage: List of Olympic medalists in figure skating\nSummary: Figure skating has been part of the Olympic Games since 1908 and has been included in 26 Olympic Games. There have been 286 medals (96 gold, 95 silver, and 95 bronze) awarded to figure skaters representing 29 representing National Olympic Committees. Six events have been contested but one, men's special figures, was discontinued after a single Olympics.\nCanadian ice dancers Tessa Virtue and Scott Moir are the only 